# Descripción de las Tablas y Columnas

## **1. MONGO01_Bioquimicos**
- **paciente_id**: Identificador único del paciente.
- **glucosa, colesterol, trigliceridos**: Niveles bioquímicos en sangre.
- **hemoglobina, leucocitos, plaquetas, creatinina**: Indicadores de salud sanguínea.

## **2. MONGO02_Clinicos**
- **paciente_id**: Identificador único del paciente.
- **diabetes, hipertension, obesidad, cancer**: Enfermedades crónicas.
- **enfermedad_cardiaca, asma, epoc**: Condiciones clínicas específicas.

## **3. MONGO03_Geneticos**
- **paciente_id**: Identificador único del paciente.
- **mut_BRCA1, mut_TP53, mut_EGFR, mut_KRAS**: Mutaciones genéticas relevantes.
- **mut_PIK3CA, mut_ALK, mut_BRAF**: Mutaciones adicionales.

## **4. MONGO04_Economicos**
- **paciente_id**: Identificador único del paciente.
- **ingresos_mensuales, gastos_salud**: Información financiera.
- **seguro_salud, deudas, tipo_empleo, ayudas_publicas**: Datos socioeconómicos.

## **5. MONGO05_Generales**
- **paciente_id**: Identificador único del paciente.
- **fumador, alcohol, actividad_fisica, vive**: Hábitos y estilo de vida.

## **6. MONGO06_Sociodemograficos**
- **paciente_id**: Identificador único del paciente.
- **edad, sexo, estado_civil**: Datos personales.
- **nivel_educativo, ocupacion, region, pais_nacimiento, codigo_postal**: Información demográfica.

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas en la salida
# Ruta del archivo
ruta_archivo = r"..\csv\raw\Csv_Todas_Columnas.csv"

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv(ruta_archivo)

# Mostrar las primeras filas para comprobar
df.head()


,paciente_id,glucosa,colesterol,trigliceridos,hemoglobina,leucocitos,plaquetas,creatinina,diabetes,hipertension,obesidad,cancer,enfermedad_cardiaca,asma,epoc,mut_BRCA1,mut_TP53,mut_EGFR,mut_KRAS,mut_PIK3CA,mut_ALK,mut_BRAF,ingresos_mensuales,gastos_salud,seguro_salud,deudas,tipo_empleo,ayudas_publicas,fumador,alcohol,actividad_fisica,vive,edad,sexo,estado_civil,nivel_educativo,ocupacion,region,pais_nacimiento,codigo_postal
0,P1000000,"108,81","226,16","161,07","13,24","6,12","258,09","0,97",1,1,1,0,1,0,0,0,0,0,0,0,0,1,"2699,72",450,1,0,Público,1,1,1,Baja,1,69,F,Viudo,Secundaria,Empleado,Centro,Indonesia,98471
1,P1000001,"74,23","148,89","110,77","13,68","6,97","278,32","0,85",0,0,0,0,0,0,0,0,0,0,0,0,0,1,"3403,55",150,1,0,Temporal,1,0,1,Moderada,0,32,M,Soltero,Universitaria,Autónomo,Sur,Brazil,23903
2,P1000002,"88,61","177,53","91,93","12,76","7,54","245,35","1,59",0,0,0,0,0,0,1,0,0,0,0,0,0,0,"2422,41",150,1,0,Desempleado,1,1,1,Baja,1,89,M,Viudo,Primaria,Desempleado,Norte,Saint Pierre and Miquelon,20094
3,P1000003,"90,92","203,97","121,79","12,95","8,56","245,11","1,26",1,0,1,1,1,0,0,0,0,0,1,0,0,0,"1695,42",450,1,0,Temporal,1,1,1,Baja,0,78,M,Viudo,Secundaria,Autónomo,Centro,Indonesia,55575
4,P1000004,"69,17","154,33","78,34","12,34","8,2","234,44","0,88",0,0,0,0,0,0,0,0,0,0,0,0,0,0,"1709,51",150,1,0,Temporal,1,1,1,Moderada,1,38,F,Soltero,Posgrado,Autónomo,Norte,Chile,20826
